# Environment Setup

conda create --prefix D:\AI-ML\LLM\llms_course_env python=3.11 -y

conda activate D:\AI-ML\LLM\llms_course_env

conda install -c conda-forge jupyter ipykernel -y

python -m ipykernel install --user --name=llms_course_env --display-name "Python (llms_course_env)"

(D:\AI-ML\LLM\llms_course_env) D:\AI-ML\LLM>jupyter notebook

In the notebook interface, go to Kernel → Change Kernel → Python (llms_course_env)

# OpenAI Setup

In [6]:
import os
import certifi
from openai import OpenAI
import config

In [7]:
# Fix SSL verification issues
os.environ["SSL_CERT_FILE"] = certifi.where()

In [8]:
# Initialize client correctly
client = OpenAI(api_key=config.api_key)

In [9]:
def generate_text(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

### Gemini AI 

In [4]:
import os
import google.generativeai as genai
import config 

In [5]:
# Configure Gemini API
genai.configure(api_key=config.gemini_api_key)

# Initialize model
model = genai.GenerativeModel("gemini-2.0-flash")

In [40]:
def generate_text(prompt):
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": 0.8,
            "max_output_tokens": 10
        } 
    )
    return response.text.strip()

In [41]:
prompt = "Who painted Mona Lisa?"
print(generate_text(prompt))

Leonardo da Vinci painted the Mona Lisa.
